# Fixed Income HW3


Cohort1 Group1 - Noah Sutherland, Ahmed Khwaja, Mingyue Qiu, Vrushali Behara

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data=pd.read_excel('Homework_3_Data.xlsx')

1. Using the spot curve in the accompanying spreadsheet, compute the par rates for bonds with maturities of 1, 2, 3, . . ., 10 years.

In [2]:
data['par_rate']=None
for i in range(len(data)):
    data.loc[i,'par_rate']=2*(1-1*data.loc[i,'D(T)'])/sum(data.loc[:i,'D(T)'])

data[data['T'].isin(range(11))].reset_index(drop=True)[['T','par_rate']]
#plt.plot(data['T'],data['par_rate'])

,T,par_rate
0,1.0,0.030339
1,2.0,0.033504
2,3.0,0.035747
3,4.0,0.037323
4,5.0,0.038441
5,6.0,0.039265
6,7.0,0.03992
7,8.0,0.040499
8,9.0,0.041063
9,10.0,0.041644


2. Compute the DV01 for each of these 10 par bonds.

In [3]:
data['duration']=None
for i in range(len(data)):
    data.loc[i,'duration']=sum(data.loc[i,'par_rate']/2*data.loc[:i,'D(T)']*data.loc[:i,'T'])+data.loc[i,'D(T)']*data.loc[i,'T']
data['mod_dur']=data['duration']/(data['par_rate']/2+1)
data['DV01']=data['mod_dur']*100*0.0001
data[data['T'].isin(range(11))].reset_index(drop=True)[['T','DV01']]

,T,DV01
0,1.0,0.009777
1,2.0,0.019188
2,3.0,0.028204
3,4.0,0.036825
4,5.0,0.045067
5,6.0,0.052946
6,7.0,0.060471
7,8.0,0.067642
8,9.0,0.074453
9,10.0,0.080892


3. Compute the Macauley and modified durations for each of these 10 par bonds.

In [4]:
data[data['T'].isin(range(11))].reset_index(drop=True)[['T','duration','mod_dur']]

,T,duration,mod_dur
0,1.0,0.992521,0.97769
1,2.0,1.950982,1.918838
2,3.0,2.870774,2.820364
3,4.0,3.751211,3.682489
4,5.0,4.593355,4.506734
5,6.0,5.398591,5.294645
6,7.0,6.167829,6.047127
7,8.0,6.901201,6.764227
8,9.0,7.598125,7.445265
9,10.0,8.257624,8.08919


4. Assume that you have a $100 position in a 5-year par bond. What position in 2-year and 10-year par bonds would be needed to hedge the 5-year position against parallel shifts in the term structure? Demonstrate that your answer works by showing how price and reinvestment risk offset each other if there is an immediate one-time parallel shift in the yield curve immediately after putting on the position.

In [5]:
d1=data[data['T']==2]['mod_dur'].values[0]
d2=data[data['T']==5]['mod_dur'].values[0]
d3=data[data['T']==10]['mod_dur'].values[0]
# weight of 2yr bond
w_2=(d3-d2)/(d3-d1)
# weight of 10yr bond
w_10=1-w_2

print(f'The position in 2yrs and 10yrs par bonds should be ${round(-w_2*100,2)} and ${round(-w_10*100,2)} respectively.')

The position in 2yrs and 10yrs par bonds should be $-58.06 and $-41.94 respectively.


In [6]:
# price change = -dollar duration * price * yield change
# dollar duration of the position
-data[data['T']==2]['DV01'].values[0]*w_2-data[data['T']==10]['DV01'].values[0]*w_10+data[data['T']==5]['DV01'].values[0]

0.0

In [7]:
data['convexity']=None
for i in range(len(data)):
    data.loc[i,'convexity']=(sum(data.loc[i,'par_rate']/2*(data.loc[:i,'T']*2)*(data.loc[:i,'T']*2+1)*data.loc[:i,'D(T)'])+data.loc[i,'D(T)']*(data.loc[i,'T']*2)*(data.loc[i,'T']*2+1))/((1+data.loc[i,'par_rate']/2)**2*4)
data[data['T'].isin(range(11))].reset_index(drop=True)[['T','convexity']]

,T,convexity
0,1.0,1.440993
1,2.0,4.678763
2,3.0,9.554129
3,4.0,15.914579
4,5.0,23.620004
5,6.0,32.539475
6,7.0,42.544426
7,8.0,53.502107
8,9.0,65.27187
9,10.0,77.7057


6. Use the computed dollar durations and convexities for these bonds and compute
the price change of a 100 basis point upward and downward parallel shift in the
spot curve. Compare the price changes with the actual price change obtained by
recomputing the price of the bond from the shifted spot curve.

In [11]:
# 1 Using the computed dollar durations and convexities
data=data.assign(up100bp_1=lambda x:-x['DV01']*100+1/2*0.01**2*x['convexity']*100,
                 down100bp_1=lambda x:-x['DV01']*(-100)+1/2*(-0.01)**2*x['convexity']*100)
# 2 From the shifted spot curve
data=data.assign(spot_rate=lambda x: (pow(1/x['D(T)'],1/(2*x['T']))-1)*2,
                 up_spot_rate=lambda x: x['spot_rate']+0.01,
                 down_spot_rate=lambda x: x['spot_rate']-0.01,
                 up_DT=lambda x:1/(1+x['up_spot_rate']/2)**(2*x['T']),
                 down_DT=lambda x:1/(1+x['down_spot_rate']/2)**(2*x['T']))

data[['up100bp_2','down100bp_2']]=None
for i in range(len(data)):
    data.loc[i,'up100bp_2']=100*(sum(data.loc[i,'par_rate']/2*data.loc[:i,'up_DT'])+data.loc[i,'up_DT'])-100
    data.loc[i,'down100bp_2']=100*(sum(data.loc[i,'par_rate']/2*data.loc[:i,'down_DT'])+data.loc[i,'down_DT'])-100

data[data['T'].isin(range(11))].reset_index(drop=True)[['T','up100bp_1','down100bp_1','up100bp_2','down100bp_2']]

,T,up100bp_1,down100bp_1,up100bp_2,down100bp_2
0,1.0,-0.970485,0.984895,-0.970532,0.984943
1,2.0,-1.895444,1.942232,-1.895672,1.942462
2,3.0,-2.772593,2.868134,-2.773211,2.86876
3,4.0,-3.602916,3.762062,-3.604196,3.763362
4,5.0,-4.388634,4.624834,-4.390899,4.627142
5,6.0,-5.131948,5.457343,-5.135566,5.461045
6,7.0,-5.834405,6.259849,-5.839777,6.26537
7,8.0,-6.496716,7.031737,-6.504268,7.039531
8,9.0,-7.118905,7.771624,-7.129077,7.782164
9,10.0,-7.700662,8.477719,-7.713898,8.491485


The absolute price changes computed by using dollar durations and convexities are smaller than actual price changes.